<a href="https://colab.research.google.com/github/andrewcgaitskell/celldb/blob/master/CellMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import xlsxwriter
import xlrd
import matplotlib

# Introduction

In [3]:
from datetime import datetime
import os

In [4]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2024-09-25T19:55:03.565945


In [5]:
print(my_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z'))

2024-09-25T19:55:03.565945


In [6]:
runid = "R" + my_date.strftime('%Y%m%d%H%M%S%f%z')
runid

'R20240925195503565945'

In [7]:
dirname = os.getcwd()
print(dirname)

/home/home_user/Code/pipeline-202102/Notebooks


In [17]:
import io
filedf = pd.read_excel("Data.xlsx",sheet_name=None, header=None)
sheetnames = list(filedf.keys())
sheetnames

['AgeGenderSheet',
 'HomeSheet',
 'JobSheet',
 'MaritalDepartmentSheet',
 'DataMap',
 'TemplateValidationMap',
 'DataSet',
 'Rules',
 'Palette',
 'PickList',
 'MetaData']

In [28]:
data_json = dict()
for sn in sheetnames:
    sheet_json = filedf[sn].to_json(path_or_buf=None,
                                      orient='records',
                                      date_format='iso',
                                      double_precision=10,
                                      force_ascii=True,
                                      date_unit='ms',
                                      default_handler=None,
                                      lines=False,
                                      compression='infer',
                                      index=None, indent=None,
                                      storage_options=None, mode='w')
    data_json[sn] = sheet_json

In [46]:
data_json

{'AgeGenderSheet': '[{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":"Age Table","2":null,"3":null,"4":null},{"0":null,"1":null,"2":"Name","3":"Age","4":null},{"0":null,"1":null,"2":"Paul","3":-52,"4":null},{"0":null,"1":null,"2":"Ringo","3":22,"4":null},{"0":null,"1":null,"2":"Brian","3":89,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3":"Gender Table","4":null},{"0":null,"1":null,"2":null,"3":"Name","4":"Gender"},{"0":null,"1":null,"2":null,"3":"Paul","4":"Male"},{"0":null,"1":null,"2":null,"3":"Ringo","4":"Prefer not to say"},{"0":null,"1":null,"2":null,"3":"Brian","4":"Male"},{"0":null,"1":null,"2":null,"3":null,"4":null},{"0":null,"1":null,"2":null,"3

In [47]:
import openpyxl
import json
from openpyxl.utils.datetime import from_excel
from datetime import datetime

def is_date_format(cell):
    """Check if the cell is formatted as a date in Excel."""
    # Excel's date formats commonly contain 'd', 'y', or 'm'
    if cell.number_format.lower().find('d') != -1 or \
       cell.number_format.lower().find('y') != -1 or \
       cell.number_format.lower().find('m') != -1:
        return True
    return False

def excel_to_json(excel_file):
    # Load the workbook
    workbook = openpyxl.load_workbook(excel_file)#, data_only=True)
    sheets_data = []

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        sheet_cells = []
        
        # Store merged cells ranges
        merged_cells_ranges = sheet.merged_cells.ranges
        
        # Iterate over rows in the sheet
        for row in sheet.iter_rows(values_only=False):
            for cell in row:
                # Get the cell value and row/column IDs
                cell_value = cell.value
                row_id = cell.row
                column_id = cell.column  # This gives the numeric index for the column (1, 2, 3, etc.)

                # Check if the cell is part of a merged cell range
                for merged_range in merged_cells_ranges:
                    if cell.coordinate in merged_range:
                        # Get the top-left cell value from the merged range
                        top_left_cell = sheet[merged_range.min_row][merged_range.min_col - 1]
                        cell_value = top_left_cell.value
                        break


                cell_type = str(type(cell_value).__name__)
                
                # Handle Excel's date serial numbers
                if is_date_format(cell):
                    try:
                        # Try to convert the cell value to a date if it's a serial number
                        if isinstance(cell_value, (int, float)):
                            cell_value = from_excel(cell_value, cell.parent.epoch).isoformat()
                            cell_type = "datetime"
                        elif isinstance(cell_value, datetime):
                            # Already a datetime object, convert to isoformat
                            cell_value = cell_value.isoformat()
                            cell_type = "datetime"
                    except:
                        pass  # In case it's not convertible, leave as-is
                
                # Ensure scientific notation is properly formatted
                if isinstance(cell_value, float):
                    cell_value = '{:.8e}'.format(cell_value) if 'e' in str(cell_value) else cell_value

                # Append the cell data with row_id, column_id, value, and type
                append_this = {
                    "sheet_name" : sheet_name,
                    "row_id": row_id,
                    "column_id": column_id,
                    "value": cell_value,
                    "type": cell_type
                }
        
                sheets_data.append(append_this)
        return_data = []
        for sd in sheets_data:
            if sd['value'] != None:
                return_data.append(sd)
    
    # Convert the data to a JSON string
    return return_data

# Example usage
excel_file_path = 'Data.xlsx'
json_result = excel_to_json(excel_file_path)

# Print or use the JSON data as needed
print(json_result)




[{'sheet_name': 'AgeGenderSheet', 'row_id': 5, 'column_id': 2, 'value': 'Age Table', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 5, 'column_id': 3, 'value': 'Age Table', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 5, 'column_id': 4, 'value': 'Age Table', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 6, 'column_id': 3, 'value': 'Name', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 6, 'column_id': 4, 'value': 'Age', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 7, 'column_id': 3, 'value': 'Paul', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 7, 'column_id': 4, 'value': -52, 'type': 'int'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 8, 'column_id': 3, 'value': 'Ringo', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 8, 'column_id': 4, 'value': 22, 'type': 'int'}, {'sheet_name': 'AgeGenderSheet', 'row_id': 9, 'column_id': 3, 'value': 'Brian', 'type': 'str'}, {'sheet_name': 'AgeGenderSheet', 'row_id


# Criteria for querying

##city_criteria = 'New York'

# Querying the list of dictionaries


In [54]:
sheet_name = 'PickList'

picklists = [ entry for entry in json_result 
    ## if entry['age'] == age_criteria and entry['city'] == city_criteria
    if entry['sheet_name'] == sheet_name and entry['row_id'] == 1]

keywords = [entry['value'] for entry in picklists]

result = [ entry for entry in json_result 
    ## if entry['age'] == age_criteria and entry['city'] == city_criteria
    if entry['sheet_name'] == sheet_name and entry['column_id'] == 1]

print(keywords)


['MaritalStatus', 'HomeLocation', 'Gender']


# Get Picklist Values

Picklist are like a mini Master Data Management System

They define the acceptable values for a partcular Field in Excel

The challenge here was to check one column to a another column that contained a list.  I go there in the end.

In [ ]:
PickListCells = pd.DataFrame()
# This process uses the full cell database as the column headings are the Picklist Names
PickListCells = fullcelldb[(fullcelldb['SheetName'] == 'PickList')].copy()
# PickListCellsList = PickListCells.groupby('ColumnID')['CellValue'].apply(lambda group_series: group_series.tolist()).reset_index()

PickListCellsList = PickListCells.groupby('ColumnID').agg({'CellValue':lambda x: list(x)})

# these were odd, but glad I work them out, it is important to realise that these are operations on a number of rows, not just one
# maybe it was the axis that made the difference!
PickListCellsList['KeyWord'] = PickListCellsList.apply(lambda row : row['CellValue'][0], axis = 1)
PickListCellsList['FormulaKeyWord'] = PickListCellsList.apply(lambda row : 'PickList', axis = 1)
PickListCellsList['List'] = PickListCellsList.apply(lambda row : row['CellValue'][1:], axis = 1)
# PickListCellsList['KeyWord'] = PickListCellsList['CellValue']
#PickListCellsList= pd.DataFrame(PickListCellsList, columns=['KeyWord', 'List'])
PickListCellsList = pd.DataFrame(PickListCellsList, columns=['KeyWord', 'FormulaKeyWord', 'List'])
PickListCellsList.head()

In [ ]:
# These were attempts at creating a suitable list for the function to check to

# PickListCellsList['Result'] = PickListCellsList['KeyWord'].isin(PickListCellsList['List']).map({True: 0, False: 1})

# PickListCellsList['Result'] = PickListCellsList['KeyWord'].str.contains('|'.join(PickListCellsList['List']))
# PickListCellsList['Result'] = PickListCellsList['KeyWord'].str.contains('|'.join(x) for x in PickListCellsList['List'])
# ["".join(x) for x in lst]
# PickListCellsList['ListAsString'] = PickListCellsList['List'].str.join('-')
# PickListCellsList['ListAsTuple'] = tuple(PickListCellsList['CellValue'])


In [ ]:
#MyList = pd.DataFrame()
#MyList =  PickListCellsList
# still do not know why the following DOES NOT WORK!
# MyList['Result'] = PickListCellsList['ListAsString'].isin(PickListCellsList['KeyWord'])

# leaving this as a rememberance of the 6 hours spent on this!
# MyList['Result1'] = PickListCellsList['KeyWord'].apply(lambda x: any(i in x for i in PickListCellsList['List']))
# MyList['Result2'] = PickListCellsList['ListAsString'].str.contains(PickListCellsList['KeyWord'])
# mask = df['A'].apply(lambda x: any(i in x for i in search_list))
# PickListCellsList['Result2'] = PickListCellsList['ListAsTuple'].str.contains(PickListCellsList['KeyWord'])
# PickListCellsList['Result2'] = PickListCellsList['KeyWord'].str.contains(tuple(PickListCellsList['List']))
# PickListCellsList['ListAsTuple'].isin(PickListCellsList['KeyWord'])
# MyList['Result2'] = PickListCellsList['List'].apply(lambda x: True if (PickListCellsList.KeyWord in x).any() else False)
######### MyList['sing'] = 'Single'
# MyList['res1'] = MyList.apply(lambda ts: any(ts == 'Single'), axis=1)
# MyList['res2'] = MyList.apply(lambda ts: MyList['KeyWord'].str.contains('Single'), axis=1)
######### MyList['res3'] = MyList.apply(lambda x: x['sing'] in x['ListAsTuple'], axis=1)  ###############
######### MyList['res4'] = MyList.apply(lambda x: x['sing'] in x['List'], axis=1).map({True: 1, False: 0}) ###############
# # # # dataset.apply(lambda x: x['CellValue'] in x['PickList'], axis=1).map({True: 1, False: 0})
# data['Activity].str.contains('skin diving')
# lambda x : True if (x > 10 and x < 20) else False
#MyList.head()
# https://stackoverflow.com/questions/48460234/pandas-apply-valueerror-the-truth-value-of-a-series-is-ambigous

# Get Data Map

In [ ]:
DataMapCells = pd.DataFrame()
DataMapCells = fullcelldb[(fullcelldb['SheetName'] == 'DataMap')].copy()
DataMapCells.head()

Below extracts a Sheet from the cell database and reshapes it to it original tabular form.

In [ ]:
DataMap = pd.DataFrame()
DataMap = DataMapCells.pivot_table(index = ['RowID'], values = 'CellValue',columns = 'ColumnID', aggfunc = 'max')
ColumnHeadings = DataMap.iloc[0]
ColumnHeadingsList = list(ColumnHeadings)
print(ColumnHeadingsList)
ColumnHeadingsListTuple = enumerate(ColumnHeadingsList,start=1)
ColumnHeadingsListDict = dict(ColumnHeadingsListTuple)
print(ColumnHeadingsListDict)
DataMap = DataMap.rename(columns=ColumnHeadingsListDict)
DataMap = DataMap.drop([1])
DataMap.head()

# Make Rules Database

In [ ]:
RulesDF.head()

The above is the Rules definition sheet. It used KeyWords to define the checks.  Below forms a list of KeyWords that are relevant to a particular field in a DataSet

In [ ]:
rulesdb = pd.DataFrame()
rulesdb = pd.melt(RulesDF, id_vars=['RuleID', 'DataSet', 'FieldID'])
rulesdb = rulesdb.rename(columns={"variable": "RuleType", "value": "KeyWord"})
rulesdb = rulesdb[rulesdb.KeyWord.notnull()]
# df['A'] = df.apply(lambda x: x['B'] if x['A']==0 else x['A'], axis=1)
# The following adds the KeyWord Null into the Rules list - if Null is N, this means Nulls are Not Allowed
rulesdb['KeyWord'] = rulesdb.apply(lambda x: x['RuleType'] if x['KeyWord']=='N' else x['KeyWord'], axis=1)

PickListCellsList

rulesdb = pd.merge(rulesdb,PickListCellsList, on=['KeyWord'],how='left')

rulesdb.head(5)

# Add Code and Formulae to Rules Database

Palette defines the Rule Text and Rule Formula.  We have implemented rules on Alteryx, SQL, Python & Javascript

In [ ]:
palette = pd.DataFrame()
palette = pd.merge(DataSetDF, rulesdb, on=['FieldID', 'DataSet'])
palette['KeyWord'] = rulesdb.apply(lambda x: x['FormulaKeyWord'] if x['FormulaKeyWord']=='PickList' else x['KeyWord'], axis=1)
palette.head(5)

In [ ]:

palette = pd.merge(palette, PaletteDF, on=['KeyWord'],how='left')


In [ ]:
palette.head(5)

# Get Start Anchor Cells

Start Anchors are where tables are mapped from, every dataset in the spreadsheet must be anchored to a uniquely identifiable cell in a particular sheet.

In [ ]:
startanchors = pd.DataFrame()
startanchors = pd.DataFrame(DataSetDF, columns=['DataSet', 'SheetName', 'ReferenceTextStart'])
startanchors = startanchors.rename(columns={"ReferenceTextStart": "CellValue"})
startanchors = startanchors.drop_duplicates()
startanchors.head()


# Get All End Anchor Cells

End anchors are where a dataset finishes. Probably at the next Start Anchor.
This does mean that blank rows are validated and one needs to filter these out later. 

In [ ]:
endanchors = pd.DataFrame()
endanchors = pd.DataFrame(DataSetDF, columns=['DataSet', 'SheetName', 'ReferenceTextEnd'])
endanchors = endanchors.rename(columns={"ReferenceTextEnd": "CellValue"})
endanchors = endanchors.drop_duplicates()
endanchors.head()

# Find Start Anchors

Now we look for the start and finish anchor cells for each dataset.

In [ ]:
findstartanchors = pd.DataFrame()
findstartanchors = pd.merge(startanchors, fullcelldfstage, on=['SheetName', 'CellValue'])
findstartanchors = findstartanchors.rename(columns={"RowID": "AnchorRow", "ColumnID": "AnchorColumn"})
findstartanchors = pd.DataFrame(findstartanchors, columns=['DataSet', 'SheetName', 'AnchorRow', 'AnchorColumn'])
findstartanchors.head()

# Find End Anchors

Some datasets have no finish anchors as they are the last table in the sheet.

In [ ]:
findendanchors = pd.DataFrame()
findendanchors = pd.merge(endanchors, fullcelldfstage, on=['SheetName', 'CellValue'])
findendanchors = findendanchors.rename(columns={"RowID": "EndRow"})
findendanchors = findendanchors[findendanchors.CellValue.notnull()]
findendanchors = pd.DataFrame(findendanchors, columns=['DataSet', 'SheetName', 'EndRow'])
findendanchors.head()


# Join Anchors Together

Join the anchors to allow them to be joined with the data table
As stated above, some DataSets have NaN as the EndRow as there is no further data in the sheet

In [ ]:
anchors = pd.DataFrame()
anchors = pd.merge(findstartanchors, findendanchors, on=['DataSet', 'SheetName'], how='left')
anchors.head()

# Using Anchors and Dataset Create Map to Find Data

The DataSet defines where the DataSets can be found from the Start Anchor Cells
The following joins the found cells with the DataSet map.

In [ ]:
datasetmap = pd.DataFrame()
datasetmap = pd.merge(DataSetDF, anchors, on=['DataSet', 'SheetName'], how='left')
datasetmap = pd.DataFrame(datasetmap,
                          columns=['DataSet', 'SheetName', 'FieldID',
                                   'AnchorRow', 'AnchorColumn','EndRow',
                                   'RowFromReferenceText',
                                   'ColumnFromReferenceText','FieldPosition'])


## Show Dataset Map

In [ ]:
datasetmap.head()

# Find where Data Starts and what Columns Fields are In

One needs to now calculate the actual rows and columns of where the DataSet is.
The above table is relative to the Anchor Cells

In [ ]:
datasetmap['StartRow'] = datasetmap['AnchorRow'] + datasetmap['RowFromReferenceText']
datasetmap['FieldColumn'] = datasetmap['AnchorColumn'] + datasetmap['ColumnFromReferenceText']
dataset = pd.DataFrame()
dataset = pd.merge(fullcelldfstage, datasetmap, on=['SheetName'], how='left')
## the above may seem a cartesian join and may need to be looked at if this slows the process down

# Filter Out Data after End Anchor Row

In [ ]:
dataset = dataset[(dataset.RowID >= dataset.StartRow) &
(dataset.ColumnID == dataset.FieldColumn) &
((dataset.RowID < dataset.EndRow) | pd.isnull(dataset.EndRow))]
# the | above is an OR as if there is no EndRow for the dataset you want to
# allow the data through

In [ ]:
palette.head(5)

# Create a CellValue as Number for use in Formulae

In [ ]:
dataset = pd.DataFrame(dataset,columns=['DataSet', 'SheetName', 'FieldID', 'RowID', 'ColumnID','CellValue', 'CellType'])
dataset['CellValueAsNumber'] = pd.to_numeric(dataset['CellValue'], errors='coerce')
dataset.head(5)

# Create a List of Checks to do

In [ ]:
todolist = pd.DataFrame()
todolist = pd.merge(dataset, palette, on=['DataSet', 'FieldID'])
todolist.head(5)

# Create a List of distinct formula to apply

In [ ]:
formulas = pd.DataFrame()
formulas = pd.DataFrame(todolist, columns=['KeyWord','PythonFormula'])
formulas = formulas.drop_duplicates()
formulas.head(10)

# Prepare Results Table

In [ ]:
global bigresults
littletodolist = pd.DataFrame()
littletodolistworking = pd.DataFrame()
bigresults = pd.DataFrame()


In [ ]:
# this was an important test as if you do not have copy you get an error as it believes
# you are trying to change a view
#littletodolist = todolist[(todolist.KeyWord == 'PickList')].copy()
#littletodolist.head(100)
#filterdf = pd.DataFrame()
#filterdf = todolist[(todolist.KeyWord == 'PickList')].copy()
#filterdf.head()

In [ ]:
# littletodolist.loc[littletodolist.KeyWord == 'Text', 'Result'] = np.where(littletodolist['CellType'] == 'Text', 1, 0)

# Loop through each Formula and apply to the To Do List

In [ ]:
todolist = todolist.rename(columns={'CellValue_x': 'CellValue'})
##todolist.columns

In [ ]:
for index, row in formulas.iterrows():
  # print(row['KeyWord'],row['PythonFormula'])

  pf = row['PythonFormula']
  kw = row['KeyWord']
 
  try:
      pf1 = pf.replace("dataset", "littletodolist")
  except:
      pf1 = "1"
  littletodolist = todolist[(todolist.KeyWord == kw)].copy()
  formula2do = "littletodolist['Result'] = " + pf1
  print(kw)
  print(formula2do)
  exec(formula2do)
  ##print(littletodolist)
  ##try:
  ##bigresults = pd.concat([bigresults, littletodolist], ignore_index=True).drop_duplicates().reset_index(drop=True)
  bigresults = pd.concat([bigresults, littletodolist]) ##, ignore_index=True).drop_duplicates().reset_index(drop=True)
  ##except:
  #  bigresults = bigresults

In [ ]:
littletodolist['Result'] = np.where(littletodolist['CellType'] == 'Text', 1, 0)

# Add a sequence to the Results for use in Reporting

In [ ]:
bigresults['ResultID'] = bigresults.index

In [ ]:
bigresults.head(5)

# Narrow down Results to useful columnns

In [ ]:
usefulresults = pd.DataFrame()
usefulresults = pd.DataFrame(bigresults, columns=['DataSet','FieldID','RowID','ColumnID','CellValue','CellType','CheckPerformed', 'Result'])
usefulresults.head()

# Summarise Results

In [ ]:
resultssummary = pd.DataFrame()
# index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
# resultssummary.head()

resultssummary = bigresults[['DataSet','FieldID','CheckPerformed', 'Result', 'ResultID']].groupby(['DataSet', 'FieldID', 'CheckPerformed']).agg(['count'])
resultssummary.head(5)

# Pivot Results

In [ ]:
# rownamearray = pd.array(['DataSet','FieldID'], dtype=str)
# newct = pd.crosstab(bigresults.DataSet,bigresults.FieldID,bigresults.CheckPerformed,bigresults.Result,rownames=rownamearray, colnames=['Result'])

newct = bigresults.pivot_table(index = ['DataSet','FieldID','CheckPerformed'], values = 'ResultID',columns = 'Result', aggfunc = 'count')
newct.head()

# Plot Stack Bar

## Show Stack

In [ ]:
newct.plot(kind='barh', stacked=True)